In [3]:
from tf_agents.environments import suite_gym

env = suite_gym.load("Breakout-v4")
env

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.8; Detected an installation of version 2.0.0. Please upgrade TensorFlow to proceed.

In [ ]:
from tf_agents.environments.wrappers import ActionRepeat

repeating_env = ActionRepeat(env, times=4)